<a href="https://colab.research.google.com/github/farhanhubble/discover-drl/blob/master/Rediscovering_RL_Notebook_0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
agent = '⚽'
opponent = '👕'
goal = '🥅'

arena = [['⚽', ' ' , '👕', ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , '🥅']]

In [3]:
import numpy as np

class Foolsball(object):

  def __to_state__(self,row,col):
    """Convert from integer state to indices (row,col)."""
    return row*self.n_cols + col

  def __to_indices__(self, state):
    """Convert indices(row,col) to state (single integer)."""
    row = state // self.n_cols
    col = state % self.n_cols
    return row,col

  def __deserialize__(self,map:list,agent:str,opponent:str, goal:str):
    """Convrt a string representation of a map into a 2D numpy array
    Param map: list of lists of strings representing the player, opponents and goal.
    Param agent: string representing the agent on the map 
    Param opponent: string representing every instance of an opponent player
    Param goal: string representing the location of the goal on the map
    """
    ## Capture dimensions and map.
    self.n_rows = len(map)
    self.n_cols = len(map[0])
    self.n_states = self.n_rows * self.n_cols 
    self.map = np.asarray(map)

    ## Store string representations for printing the map, etc.
    self.agent_repr = agent
    self.opponent_repr  = opponent
    self.goal_repr = goal

    ## Find initial state, the desired goal state and the state of the opponents. 
    self.init_state = None
    self.goal_state = None
    self.opponents_states = []

    for row in range(self.n_rows):
      for col in range(self.n_cols):
        if map[row][col] == agent:
          # Store the initial state outside the map.
          # This helps in quickly resetting the game to the initial state and
          # also simplifies printing the map independent of the agent's state. 
          self.init_state = self.__to_state__(row,col)
          self.map[row,col] = ' ' 
        
        elif map[row][col] == opponent:
          self.opponents_states.append(self.__to_state__(row,col))

        elif map[row][col] == goal:
          self.goal_state = self.__to_state__(row,col)

    assert self.init_state is not None, print(f"Map {map} does not specify an agent {agent} location")
    assert self.goal_state is not None,  print(f"Map {map} does not specify a goal {goal} location")
    assert self.opponents_states,  print(f"Map {map} does not specify any opponents {opponent} location")

    return self.init_state


  def __get_next_state_on_action__(self,state,action):
    """Return next state based on current state and action."""
    row, col = self.__to_indices__(state)
    action_to_index_delta = {'n':[-1,0], 'e':[0,+1], 'w':[0,-1], 's':[+1,0]}

    row_delta, col_delta = action_to_index_delta[action]
    new_row , new_col = row+row_delta, col+col_delta

    ## Return current state if next state is invalid
    ## The caller need to check for this
    if not(0<=new_row<self.n_rows) or not(0<=new_col<self.n_cols):
      return state  
    
    return self.__to_state__(new_row, new_col)

  
  def __init__(self,map,agent,opponent,goal):
    """Spawn the world, create variables to track state and actions."""
    # We just need to track the location of the agent (the ball)
    # Everything else is static and so a potential algorithm doesn't 
    # have to look at it. The `done` flag terminal states.
    self.state = self.__deserialize__(map,agent,opponent,goal)
    self.done = False
    self.actions = ['n','e','w','s']

    # Build a trnasition table (dict of dicts), mapping every (state,action)
    # pair to the next state. This defines the one-step dynamics of the 
    # environment 
    self.transitions = self.__install_transition_table__()


    # Set up the rewards
    self.rewards = {'unmarked':-1, 'opponent':-5, 'outside':-1, 'goal':+5}

  
  def __install_transition_table__(self):
    ## Create a dictionary of dictionaries that can map every (state,action) pair
    ## to the corresponding next state.
    transitions = {s:{a:None for a in self.actions} for s in range(self.n_states)}
    for s in range(self.n_states):
      for a in self.actions:
        transitions[s][a] = self.__get_next_state_on_action__(s,a)


  def reset(self):
    """Reset the environment to its initial state."""
    # There's really just two things we need to reset: the state, which should
    # be reset to the initial state, and the `done` flag which should be 
    # cleared to signal that we are not in a terminal state anymore, even if we 
    # were earlier. 
    self.state = self.init_state
    self.done  = False
    return self.state

  
  def step(self,action):
    """Simulate state transition based on current state and action received."""
    assert not self.done, \
    print(f'You cannot call step() in a terminal state({self.state}). Check the "done" flag before calling step() to avoid this.')
    next_state = self.__get_next_state_on_action__(self.state, action)

    ## Transition rejected due to illegal action (move)
    if next_state == self.state:
      reward = self.rewards['outside']
      done = False
    
    ## Goal!
    elif next_state == self.goal_state:
      reward = self.rewards['goal']
      done = True
    
    ## Ran into opponent. 
    elif next_state in self.opponents_states:
      reward = self.rewards['opponent']
      done = True

    ## Made a safe and valid move.   
    else:
      reward = self.rewards['unmarked']
      done = False

    self.state, self.done = next_state, done
    return next_state, reward, done


  def render(self):
    """Pretty-print the environment and agent."""
    for row in range(self.map.shape[0]):
      for col in range(self.map.shape[1]):
        ## Goal location.
        if (row,col) == self.__to_indices__(self.goal_state):
          if self.state == self.goal_state:
            print(f' 🏁 ',end="")
          else:
            print(f' {self.goal_repr}',end="")
        
        ## One of the opponent positions.
        elif self.__to_state__(row,col) in self.opponents_states:
          if (row,col) == self.__to_indices__(self.state): 
            print(' ❗ ',end='')
          else:
            print(f' {self.opponent_repr}',end="")

        ## Agent's current position.
        elif (row,col) == self.__to_indices__(self.state):
          if self.state == self.goal_state:
            print(f' 🏁 ',end="")
          else:
            print(f'{self.agent_repr} ',end="")    

        ## Unoccupied position.
        else:
          print(' + ',end="") 
      
      print('\n') 



In [4]:
foolsball = Foolsball(arena, agent, opponent, goal)

In [ ]:
foolsball.render()

In [ ]:
while True:
  try:
    act = input('>>')

    if act in foolsball.actions:
      print(foolsball.step(act))
      print()
      foolsball.render()
    elif act == 'r':
      print(foolsball.reset())
      print()
      foolsball.render()
    elif act == 'x':
      break
    else:
      print(f'Invalid input:{act}')
  except Exception as e:
    print(e)